In [1]:
import ase
import ase.io
import numpy as np
import math as m
import copy

In [45]:
directory = "./All_v2_xyz/"

In [46]:
def writexyz(atomtypes,masses,coords,filename,format = "xyz"):
    atoms = ase.Atoms(symbols = atomtypes,positions = coords,masses = masses,cell = [20,20,20])
    #print(atoms.numbers)
    ase.io.write(filename,atoms,format = format)

In [47]:
#https://mathworld.wolfram.com/EulerAngles.html 
def rotation_matrix(phi,theta,psi):
    A = np.zeros((3,3))
    A[0,0] = m.cos(psi)*m.cos(phi) - m.cos(theta)*m.sin(phi)*m.sin(psi)
    A[0,1] = m.cos(psi)*m.sin(phi) + m.cos(theta)*m.cos(phi)*m.sin(psi)
    A[0,2] = m.sin(psi)*m.sin(theta)
    A[1,0] = -m.sin(psi)*m.cos(phi) - m.cos(theta)*m.sin(phi)*m.cos(psi)
    A[1,1] = -m.sin(psi)*m.sin(phi) + m.cos(theta)*m.cos(phi)*m.cos(psi)
    A[1,2] = m.cos(psi)*m.sin(theta)
    A[2,0] = m.sin(theta)*m.sin(phi)
    A[2,1] = -m.sin(theta)*m.cos(phi)
    A[2,2] = m.cos(theta)
    return A

In [48]:
#deltas is a 3 by 3 array that provides shifts to positions of x,y,z of each of the three atoms
#theta is the theta (in xy plane rotation of the whole molecule)
#phi is the phi angle (out of xy plane rotation of the whole molecule)
#carbon center is the reference position to build the molecule off of based on the carbon.
#Atom order is expected to be C, O in +x, O in -x by default.
#bondangle is in degrees and represents how to bend the third atom with respect to the second
def create_trimer(carboncenter,bond_length,bond_angle,deltas,alignment_angle,phi,theta,psi):
    
    #atomtypes = ["C","O","O"]
    #masses = [12,16,16]
    coords = np.zeros((3,3))
    #Set carbon positions
    #print(carboncenter[0])
    for i in range(3):
        coords[0,i] = carboncenter[i]+deltas[0,i]
        #print(carboncenter[i]+deltas[0,i])
    #Set oxygen1 positions
    for i in range(3):
        coords[1,i] = carboncenter[i]+deltas[1,i]
    coords[1,0] += bond_length
    A_alignment = rotation_matrix(0,0,m.radians(alignment_angle))
    coords[1] = np.matmul(A_alignment,coords[1])
    #Set oxygen2 positions
    for i in range(3):
        coords[2,i] = carboncenter[i]+deltas[2,i]
    coords[2,0] += bond_length
    
    A_bond = rotation_matrix(m.radians(bond_angle),0,0)
    coords[2] = np.matmul(A_bond,coords[2])#+= (bond_length/2.0)*m.sin(m.radians(bondangle))
    
    #Rotate the whole molecule based on theta and phi, around the carbon:
    tempshift = copy.deepcopy(coords[0])
    #Shift to zero
    coords -= tempshift
    print("shifted coords",coords,tempshift)
    #Phi is rotation around z axis: https://mathworld.wolfram.com/EulerAngles.html
    #theta is rotation around x axis
    # psi is rotation around y axis
    A_mol = rotation_matrix(phi,theta,psi)
    #We only have to rotate the two oxygens
    for i in range(1,3):
        coords[i] = np.matmul(A_mol,coords[i])#coords[i,0]*m.cos(theta) - coords[i,1]*m.sin(theta)
        #coords[i] = #coords[i,0]*m.sin(theta) + coords[i,1]*m.cos(theta)
    
    #Unshift
    coords += tempshift
    
    
    return coords

def create_dimer(carboncenter,bond_length,bond_angle,deltas,alignment_angle,phi,theta,psi):
    
    #atomtypes = ["C","O","O"]
    #masses = [12,16,16]
    coords = np.zeros((2,3))
    #Set carbon positions
    #print(carboncenter[0])
    for i in range(3):
        coords[0,i] = carboncenter[i]+deltas[0,i]
        #print(carboncenter[i]+deltas[0,i])
    
    #Set oxygen1 positions
    for i in range(3):
        coords[1,i] = carboncenter[i]+deltas[1,i]
    coords[1,0] += bond_length
    A_alignment = rotation_matrix(0,0,m.radians(alignment_angle))
    coords[1] = np.matmul(A_alignment,coords[1])


    #Rotate the whole molecule based on theta and phi, around the carbon:
    tempshift = copy.deepcopy(coords[0])
    #Shift to zero
    coords -= tempshift
    print("shifted coords",coords,tempshift)
    #Phi is rotation around z axis: https://mathworld.wolfram.com/EulerAngles.html
    #theta is rotation around x axis
    # psi is rotation around y axis
    A_mol = rotation_matrix(phi,theta,psi)
    #We only have to rotate the two oxygens
    for i in range(1,2):
        coords[i] = np.matmul(A_mol,coords[i])#coords[i,0]*m.cos(theta) - coords[i,1]*m.sin(theta)
        #coords[i] = #coords[i,0]*m.sin(theta) + coords[i,1]*m.cos(theta)
    
    #Unshift
    coords += tempshift
    
    
    return coords

In [49]:
filename = directory+"default_CO2.extxyz"
bond_length = 1.162 #Angstroms, https://cccbdb.nist.gov/exp2x.asp?casno=124389
bond_angle = 180.0
atomtypes = ["C","O","O"]
masses = [12,16,16]

coords = create_trimer([0,0,0],bond_length,bond_angle, np.zeros((3,3)),0,0,0,0)
print(atomtypes,masses,coords)
writexyz(atomtypes,masses,coords,filename)

shifted coords [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.16200000e+00  0.00000000e+00  0.00000000e+00]
 [-1.16200000e+00 -1.42303958e-16  0.00000000e+00]] [0. 0. 0.]
['C', 'O', 'O'] [12, 16, 16] [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.16200000e+00  0.00000000e+00  0.00000000e+00]
 [-1.16200000e+00 -1.42303958e-16  0.00000000e+00]]


In [50]:
filename = directory+"default_H2O.extxyz" #https://cccbdb.nist.gov/expgeom2.asp?casno=7732185&charge=0
bond_length = 0.958
bond_angle = 104.4776
atomtypes = ["O","H","H"]
masses = [16,1,1]

coords = create_trimer([0,0,0],bond_length,bond_angle, np.zeros((3,3)),0,0,0,0)
print(atomtypes,masses,coords)
writexyz(atomtypes,masses,coords,filename)

shifted coords [[ 0.          0.          0.        ]
 [ 0.958       0.          0.        ]
 [-0.23950142 -0.92757914  0.        ]] [0. 0. 0.]
['O', 'H', 'H'] [16, 1, 1] [[ 0.          0.          0.        ]
 [ 0.958       0.          0.        ]
 [-0.23950142 -0.92757914  0.        ]]


In [51]:
filename = directory+"default_N2.extxyz" #https://cccbdb.nist.gov/exp2x.asp?casno=7727379
bond_length = 1.098
bond_angle = 180.0
atomtypes = ["N","N"]
masses = [14,14]

coords = create_dimer([0,0,0],bond_length,bond_angle, np.zeros((2,3)),0,0,0,0)
print(atomtypes,masses,coords)
writexyz(atomtypes,masses,coords,filename)

shifted coords [[0.    0.    0.   ]
 [1.098 0.    0.   ]] [0. 0. 0.]
['N', 'N'] [14, 14] [[0.    0.    0.   ]
 [1.098 0.    0.   ]]


## Exciting tests

In [52]:
squish_array = [-0.5,-0.45,-0.4,-0.35,-0.3,-0.25,-0.2,-0.15,-0.1,-0.05,-0.025,-0.01,-0.001,0.0,0.001,0.01,0.025,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5]

In [53]:
#Squishing/stretching the bond:
for squish in squish_array:
    #direction = "+"
    #if squish < 0:
    #    direction = "-"
    #print(str(squish).split(".")[-1])
    filename = directory+"bond_delta_"+str(squish*1000.0).split(".")[0]+"_CO2.extxyz"
    bond_length = 1.162 #Angstroms, https://cccbdb.nist.gov/exp2x.asp?casno=124389
    bond_angle = 180.0
    atomtypes = ["C","O","O"]
    masses = [12,16,16]
    deltas = np.zeros((3,3))
    #Add a delta along the x coordinate of bond
    deltas[1,0] = squish
    deltas[2,0] = squish

    coords = create_trimer([0,0,0],bond_length,bond_angle,deltas,0,0,0,0)
    print("Squish value",squish,atomtypes,masses,coords)
    writexyz(atomtypes,masses,coords,filename)

shifted coords [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.62000000e-01  0.00000000e+00  0.00000000e+00]
 [-6.62000000e-01 -8.10716181e-17  0.00000000e+00]] [0. 0. 0.]
Squish value -0.5 ['C', 'O', 'O'] [12, 16, 16] [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.62000000e-01  0.00000000e+00  0.00000000e+00]
 [-6.62000000e-01 -8.10716181e-17  0.00000000e+00]]
shifted coords [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 7.12000000e-01  0.00000000e+00  0.00000000e+00]
 [-7.12000000e-01 -8.71948521e-17  0.00000000e+00]] [0. 0. 0.]
Squish value -0.45 ['C', 'O', 'O'] [12, 16, 16] [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 7.12000000e-01  0.00000000e+00  0.00000000e+00]
 [-7.12000000e-01 -8.71948521e-17  0.00000000e+00]]
shifted coords [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 7.62000000e-01  0.00000000e+00  0.00000000e+00]
 [-7.62000000e-01 -9.33180861e-17  0.00000000e+00]] [0. 0. 0.]
Squish value -0.4 ['C', 'O', 'O'] [12, 16, 16] [[ 0.00000

In [54]:
#Squishing/stretching the bond:
for squish in squish_array:
    #direction = "+"
    #if squish < 0:
    #    direction = "-"
    print(str(squish).split(".")[-1])
    filename = directory+"bond_delta_"+str(squish*1000.0).split(".")[0]+"_H2O.extxyz"
    bond_length = 0.958
    bond_angle = 104.4776
    atomtypes = ["O","H","H"]
    masses = [16,1,1]
    deltas = np.zeros((3,3))
    #Add a delta along the x coordinate of bond
    deltas[1,0] = squish
    deltas[2,0] = squish

    coords = create_trimer([0,0,0],bond_length,bond_angle,deltas,0,0,0,0)
    print("Squish value",squish,atomtypes,masses,coords)
    writexyz(atomtypes,masses,coords,filename)

5
shifted coords [[ 0.          0.          0.        ]
 [ 0.458       0.          0.        ]
 [-0.11450068 -0.44345642  0.        ]] [0. 0. 0.]
Squish value -0.5 ['O', 'H', 'H'] [16, 1, 1] [[ 0.          0.          0.        ]
 [ 0.458       0.          0.        ]
 [-0.11450068 -0.44345642  0.        ]]
45
shifted coords [[ 0.          0.          0.        ]
 [ 0.508       0.          0.        ]
 [-0.12700075 -0.49186869  0.        ]] [0. 0. 0.]
Squish value -0.45 ['O', 'H', 'H'] [16, 1, 1] [[ 0.          0.          0.        ]
 [ 0.508       0.          0.        ]
 [-0.12700075 -0.49186869  0.        ]]
4
shifted coords [[ 0.          0.          0.        ]
 [ 0.558       0.          0.        ]
 [-0.13950083 -0.54028096  0.        ]] [0. 0. 0.]
Squish value -0.4 ['O', 'H', 'H'] [16, 1, 1] [[ 0.          0.          0.        ]
 [ 0.558       0.          0.        ]
 [-0.13950083 -0.54028096  0.        ]]
35
shifted coords [[ 0.          0.          0.        ]
 [ 0.608      

In [55]:
#Squishing/stretching the bond:
for squish in squish_array:
    #direction = "+"
    #if squish < 0:
    #    direction = "-"
    print(str(squish).split(".")[-1])
    filename = directory+"bond_delta_"+str(squish*1000.0).split(".")[0]+"_N2.extxyz"
    bond_length = 1.098
    bond_angle = 180.0
    atomtypes = ["N","N"]
    masses = [14,14]
    deltas = np.zeros((2,3))
    #Add a delta along the x coordinate of bond
    deltas[1,0] = squish
    #deltas[2,0] = squish

    coords = create_dimer([0,0,0],bond_length,bond_angle,deltas,0,0,0,0)
    print("Squish value",squish,atomtypes,masses,coords)
    writexyz(atomtypes,masses,coords,filename)

5
shifted coords [[0.    0.    0.   ]
 [0.598 0.    0.   ]] [0. 0. 0.]
Squish value -0.5 ['N', 'N'] [14, 14] [[0.    0.    0.   ]
 [0.598 0.    0.   ]]
45
shifted coords [[0.    0.    0.   ]
 [0.648 0.    0.   ]] [0. 0. 0.]
Squish value -0.45 ['N', 'N'] [14, 14] [[0.    0.    0.   ]
 [0.648 0.    0.   ]]
4
shifted coords [[0.    0.    0.   ]
 [0.698 0.    0.   ]] [0. 0. 0.]
Squish value -0.4 ['N', 'N'] [14, 14] [[0.    0.    0.   ]
 [0.698 0.    0.   ]]
35
shifted coords [[0.    0.    0.   ]
 [0.748 0.    0.   ]] [0. 0. 0.]
Squish value -0.35 ['N', 'N'] [14, 14] [[0.    0.    0.   ]
 [0.748 0.    0.   ]]
3
shifted coords [[0.    0.    0.   ]
 [0.798 0.    0.   ]] [0. 0. 0.]
Squish value -0.3 ['N', 'N'] [14, 14] [[0.    0.    0.   ]
 [0.798 0.    0.   ]]
25
shifted coords [[0.    0.    0.   ]
 [0.848 0.    0.   ]] [0. 0. 0.]
Squish value -0.25 ['N', 'N'] [14, 14] [[0.    0.    0.   ]
 [0.848 0.    0.   ]]
2
shifted coords [[0.    0.    0.   ]
 [0.898 0.    0.   ]] [0. 0. 0.]
Squish valu

In [56]:
nudge_array = [-45.0,-30.0,-15.0,-12.5,-10.0,-7.5,-5,-2.5,-1.0,-0.1,-0.05,-0.025,-0.01,0.0,0.01,0.025,0.05,0.1,1.0,2.5,5.0,7.5,10.0,12.5,15.0,30.0,45.0]

In [57]:
# rotating one of the atoms perpendicular to the bond plane:
for nudge in nudge_array:
    #direction = "+"
    #if nudge < 0:
    #    direction = "-"
    print(str(nudge).split(".")[-1])
    filename = directory+"alignment_angle_"+str(nudge*1000.0).split(".")[0]+"_CO2.extxyz"
    bond_length = 1.162 #Angstroms, https://cccbdb.nist.gov/exp2x.asp?casno=124389
    bond_angle = 180.0
    atomtypes = ["C","O","O"]
    masses = [12,16,16]
    deltas = np.zeros((3,3))
    #Add a nudge angle along the perpendicular angle to the bond to one atom (adding a bend/twist)

    coords = create_trimer([0,0,0],bond_length,bond_angle,deltas,nudge,0,0,0)
    print("Nudge value",nudge,atomtypes,masses,coords)
    writexyz(atomtypes,masses,coords,filename)

0
shifted coords [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 8.21658080e-01  8.21658080e-01  0.00000000e+00]
 [-1.16200000e+00 -1.42303958e-16  0.00000000e+00]] [0. 0. 0.]
Nudge value -45.0 ['C', 'O', 'O'] [12, 16, 16] [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 8.21658080e-01  8.21658080e-01  0.00000000e+00]
 [-1.16200000e+00 -1.42303958e-16  0.00000000e+00]]
0
shifted coords [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.00632152e+00  5.81000000e-01  0.00000000e+00]
 [-1.16200000e+00 -1.42303958e-16  0.00000000e+00]] [0. 0. 0.]
Nudge value -30.0 ['C', 'O', 'O'] [12, 16, 16] [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.00632152e+00  5.81000000e-01  0.00000000e+00]
 [-1.16200000e+00 -1.42303958e-16  0.00000000e+00]]
0
shifted coords [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.12240581e+00  3.00747730e-01  0.00000000e+00]
 [-1.16200000e+00 -1.42303958e-16  0.00000000e+00]] [0. 0. 0.]
Nudge value -15.0 ['C', 'O', 'O'] [12, 16, 16] [[ 0.

In [58]:
# rotating one of the atoms perpendicular to the bond plane:
for nudge in nudge_array:
    #direction = "+"
    #if nudge < 0:
    #    direction = "-"
    print(str(nudge).split(".")[-1])
    filename = directory+"alignment_angle_"+str(nudge*1000.0).split(".")[0]+"_H2O.extxyz"
    bond_length = 0.958
    bond_angle = 104.4776
    atomtypes = ["O","H","H"]
    masses = [16,1,1]
    deltas = np.zeros((3,3))
    #Add a delta along the x coordinate of bond
    #deltas[1,1] = nudge
    

    coords = create_trimer([0,0,0],bond_length,bond_angle,deltas,nudge,0,0,0)
    print("Nudge value",nudge,atomtypes,masses,coords)
    writexyz(atomtypes,masses,coords,filename)

0
shifted coords [[ 0.          0.          0.        ]
 [ 0.6774083   0.6774083   0.        ]
 [-0.23950142 -0.92757914  0.        ]] [0. 0. 0.]
Nudge value -45.0 ['O', 'H', 'H'] [16, 1, 1] [[ 0.          0.          0.        ]
 [ 0.6774083   0.6774083   0.        ]
 [-0.23950142 -0.92757914  0.        ]]
0
shifted coords [[ 0.          0.          0.        ]
 [ 0.82965234  0.479       0.        ]
 [-0.23950142 -0.92757914  0.        ]] [0. 0. 0.]
Nudge value -30.0 ['O', 'H', 'H'] [16, 1, 1] [[ 0.          0.          0.        ]
 [ 0.82965234  0.479       0.        ]
 [-0.23950142 -0.92757914  0.        ]]
0
shifted coords [[ 0.          0.          0.        ]
 [ 0.92535694  0.24794865  0.        ]
 [-0.23950142 -0.92757914  0.        ]] [0. 0. 0.]
Nudge value -15.0 ['O', 'H', 'H'] [16, 1, 1] [[ 0.          0.          0.        ]
 [ 0.92535694  0.24794865  0.        ]
 [-0.23950142 -0.92757914  0.        ]]
5
shifted coords [[ 0.          0.          0.        ]
 [ 0.93529157  0.

In [59]:
# rotating one of the atoms perpendicular to the bond plane:
for nudge in nudge_array:
    #direction = "+"
    #if nudge < 0:
    #    direction = "-"
    print(str(nudge).split(".")[-1])
    filename = directory+"alignment_angle_"+str(nudge*1000.0).split(".")[0]+"_N2.extxyz"
    bond_length = 1.098
    bond_angle = 180.0
    atomtypes = ["N","N"]
    masses = [14,14]
    deltas = np.zeros((2,3))
    #Add a delta along the x coordinate of bond
    #deltas[1,1] = nudge
    

    coords = create_dimer([0,0,0],bond_length,bond_angle,deltas,nudge,0,0,0)
    print("Nudge value",nudge,atomtypes,masses,coords)
    writexyz(atomtypes,masses,coords,filename)

0
shifted coords [[0.         0.         0.        ]
 [0.77640325 0.77640325 0.        ]] [0. 0. 0.]
Nudge value -45.0 ['N', 'N'] [14, 14] [[0.         0.         0.        ]
 [0.77640325 0.77640325 0.        ]]
0
shifted coords [[0.         0.         0.        ]
 [0.95089589 0.549      0.        ]] [0. 0. 0.]
Nudge value -30.0 ['N', 'N'] [14, 14] [[0.         0.         0.        ]
 [0.95089589 0.549      0.        ]]
0
shifted coords [[0.         0.         0.        ]
 [1.06058656 0.28418331 0.        ]] [0. 0. 0.]
Nudge value -15.0 ['N', 'N'] [14, 14] [[0.         0.         0.        ]
 [1.06058656 0.28418331 0.        ]]
5
shifted coords [[0.         0.         0.        ]
 [1.07197302 0.2376507  0.        ]] [0. 0. 0.]
Nudge value -12.5 ['N', 'N'] [14, 14] [[0.         0.         0.        ]
 [1.07197302 0.2376507  0.        ]]
0
shifted coords [[0.         0.         0.        ]
 [1.08131891 0.1906657  0.        ]] [0. 0. 0.]
Nudge value -10.0 ['N', 'N'] [14, 14] [[0.         